In [1]:
#################################
#           battery EV          #
#################################

__version__ = '0.3'

import pandas as pd
import numpy as np
from datetime import timedelta


In [2]:
class price_profiles:
    def __init__(self,request_status):
        self.request_status = request_status
    
    def getProfiles(self):
        import_price1 = pd.read_csv('data/octopus_imprt_price_southern.csv', index_col=0,parse_dates=True,header = 0)
        export_price1 = pd.read_csv('data/octopus_export_price_southern.csv', index_col=0,parse_dates=True,header = 0)

        start_time1 = "2019-1-31 00:00:00"
        end_time1 = "2020-1-30 22:30:00"
        
        start_time2 = "2018-1-31 00:00:00"
        end_time2 = "2019-1-30 22:30:00"
        
        import_price2 =import_price1.loc[start_time1:end_time1]
        export_price2 =export_price1.loc[start_time2:end_time2]
        
        import_price3 = import_price2.between_time('09:00:00','16:30:00')
        export_price3 = export_price2.between_time('09:00:00','16:30:00')
        
        for column in import_price3 [['Import Rate (p/kWh)']]:
            import_price = import_price3[column] 
            import_price_value = import_price.values
            
        for column in export_price3 [['Outgoing Agile Rate (p/kWh)']]:
            export_price = export_price3[column]
            export_price_value = export_price.values 
            
        return import_price_value, export_price_value 

In [3]:
class building_profiles:
    def __init__(self,request_status):
        self.request_status = request_status
        
        
    def getProfiles(self):
        excess_profile1 = pd.read_csv('data/Export_vector.csv', index_col=0, parse_dates=True)  # W/kWp
        excess_profile2 = excess_profile1.resample('0.5H').mean()
        excess_profile3 = excess_profile2.append(pd.DataFrame({excess_profile2.columns[0]: np.nan},
                                              index=[(excess_profile2.index[-1] +
                                                      timedelta(minutes=30))]))
        excess_profile4 = excess_profile3.interpolate()
        excess_profile5 = excess_profile4.between_time('09:00:00','16:30:00')
        excess_vector = excess_profile5.values


        import_profile1 = pd.read_csv('data/Import_vector.csv', index_col=0,parse_dates=True)  # W/kWp
        import_profile2 = import_profile1.resample('0.5H').mean()
        import_profile3 = import_profile2.append(pd.DataFrame({import_profile2.columns[0]: np.nan},
                                              index=[(import_profile2.index[-1] +timedelta(minutes=30))]))
        import_profile4 = import_profile3.interpolate()
        import_profile5 = import_profile4.between_time('09:00:00','16:30:00')
        import_vector = import_profile5.values
     
        return import_vector,excess_vector
        

In [4]:
class control_class:
    def __init__(self,request_status):
        self.request_status = request_status
        
    def signalGen(self,import_price_value,export_price_value, max_buy_price, min_sell_price,import_vector,excess_vector):     
        T = len(import_price_value) 
        signal = np.zeros((T, 1))
        
        if self.request_status == 0:  
            for time,price in enumerate(import_price_value):
                if price < max_buy_price:   # Charge/Buys when price < Maximum_Buy_Price
                    signal[time] = 1
                else:
                    signal[time] = 0  # Set all other cases to "do nothing" first
                    
            for time,price in enumerate(export_price_value):
                if price > min_sell_price :  # Discharge/Sells when price > Minimum_Sell_Price 
                    signal[time] = -1 # Corrects some cases to sell electricity 
                
        elif self.request_status == 1:       
            for time,excess_amount in enumerate(excess_vector):
                if excess_amount > 0:   # Charge the batteries for free when excess generation 
                    signal[time] = 1
                else:
                    signal[time] = 0  # Set all other cases to "do nothing" first
                    
            for time,import_amount in enumerate(import_vector):
                if import_amount > 0 :  # Discharge when building in need for more electricity  
                    signal[time] = -1 # Corrects some cases to discharge
                
        return signal

In [5]:
class EV_Component:
    def __init__(self):
        self.dispatch_type = "EV Component"
        self.capacity = 0
        self.install_cost = 0
        self.depreciation_cost = 0
        self.lifetime = 15    # Assume all components have a lifespan of 15 years 

In [30]:
class EV_battery(EV_Component):
    def __init__(self, capacity, power, efficiency, dt, install_cost, signal):
        super().__init__()
        self.asset_type = 'EV Battery'
        self.capacity = capacity
        self.power = power * dt   # Convert kW to kWh
        self.eff = efficiency
        self.soc = np.ones(signal.size) * self.capacity
        self.install_cost = install_cost * 100  # p/kWh
        self.depreciation_cost = install_cost/self.lifetime
        self.signal = signal
        
    def getOutput(self, signal,request_status,import_vector,excess_vector,energy_reserved):
        T = len(signal)
        output = np.zeros((T, 1))   # zero array of T rows, 1 column
        
        if request_status == 0:
            for time, signal_status in enumerate(signal):
                if time % 16 == 0:           # Reset the soc every morning at 9 am  
                    soc = self.capacity - 5  # Capacity minus the energy used for commuting to work 
                else:
                    soc = self.soc[time - 1]           
                if signal[time] == -1 and self.soc[time-1]>energy_reserved:       # Signal indicating selling/ Discharging 
                                                                # Whilst also reserving enough for traveling home
                    output[time] = min(self.power, self.eff*(soc-energy_reserved))
                    self.soc[time] = soc - (1/self.eff)*output[time]
                elif signal[time] == 1:  # Signal indicating buying/ Charging 
                    output[time] = max(-self.power, - (1/self.eff) * (self.capacity - soc))
                    self.soc[time] = soc - self.eff * output[time]        
                else:  # do nothing
                    self.soc[time] = soc
            
        if request_status == 1:
            for time, signal_status in enumerate(signal):
                if time % 16 == 0:           # Reset the soc every morning at 9 am  
                    soc = self.capacity - 5  # Capacity minus the energy used for commuting to work 
                else:
                    soc = self.soc[time - 1]           
                if signal[time] == -1 and self.soc[time-1]>energy_reserved:       # Signal indicating discharging
                                                                # Whilst also reserving enough for traveling home
                    output[time] = min(self.power, self.eff*(soc-energy_reserved),self.eff*import_vector[time])
                    self.soc[time] = soc -(1/self.eff)*output[time]
                elif signal[time] == 1:  # Signal indicating Charging 
                    output[time] = max(-self.power, -(1/self.eff)*(self.capacity - soc),-(1/self.eff)*excess_vector[time])
                    self.soc[time] = soc - self.eff * output[time]        
                else:  # do nothing
                    self.soc[time] = soc
                print(self.soc[time])
        return output 
    
    def getGridCost(self, output, import_price_value, export_price_value,request_status,import_vector,excess_vector):
        cost_arr_size = len(output)
        cost_array = np.zeros((cost_arr_size, 1))   # zero array of T rows, 1 column
        
        for time, output_power in enumerate(output):
            if request_status == 0:
                if output[time] > 0:    # Positive output power, selling/discharging
                    cost_array[time] = - export_price_value[time]*output[time] #sell cost is negative
                elif output[time] < 0:  # Negative output power, buying/charging
                    cost_array[time] = - import_price_value[time]*output[time] #buy cost is positive           
                elif output[time] == 0:  # No power output, do nothing 
                    cost_array[time] = 0         
                #print(cost_array[time])
                
            if request_status == 1:
                if output[time] > 0:    # Positive output power, discharging to the buildings
                    # if building demand is not met, some electricity needs to be bought from grid
                    cost_array[time] = export_price_value[time]*(import_vector[time]- output[time]) #buy cost is positive
                elif output[time] < 0:  # Negative output power, charging from the excess of the builidngs 
                    # if building excess is too much, some electricity needs to be sold to grid
                    cost_array[time] = import_price_value[time]*(excess_vector[time]- output[time]) #sell cost is negative         
                elif output[time] == 0:  # No power output, do nothing 
                    cost_array[time] = 0         
                print(cost_array[time]/100) 
                
                
                
        grid_cost = np.sum(cost_array) # in pence 
        grid_cost_GBP = grid_cost/100  # convert to pounds
        print('Grid cost: £ %3.3f' % grid_cost_GBP)
        return grid_cost_GBP
    

In [31]:
class V2G_Charger(EV_Component): 
    def __init__(self, install_cost = 0):
        super().__init__()
        self.asset_type = 'EV Charger'
        self.install_cost = install_cost * 100  # p/kWh

In [32]:
#######################
##   Main Programme  ##
#######################
dt = 60/60 # One-hour intervals 
request_status = 1 # Buildings require electricity status

max_buy_price = 4
min_sell_price = 5
energy_reserved = 5

In [33]:
# Price 
price_profiles1 = price_profiles(request_status)
import_price_value, export_price_value = price_profiles1.getProfiles()

In [34]:
# Building Profiles 
building_profiles1 = building_profiles(request_status)
import_vector1,excess_vector1 = building_profiles1.getProfiles()

In [35]:
# Control Signals
signal = control_class(request_status)
signal1 = signal.signalGen(import_price_value, export_price_value, max_buy_price, min_sell_price,import_vector1,excess_vector1)

In [36]:
# Battery
battery_capacity = 60  # kWh
battery_power = 10  # kW
battery_eff = 0.8
battery_cost = 0 
battery_site1 = EV_battery(battery_capacity,battery_power,battery_eff, dt, battery_cost,signal1)
output1 = battery_site1.getOutput(signal1,request_status,import_vector1,excess_vector1,energy_reserved)

42.5
30.0
17.5
5.0
5.0
5.0
5.0
5.0
5.0
5.0
5.0
5.0
5.0
5.0
5.0
5.0
55.0
55.0
55.0
55.0
55.0
55.0
55.0
55.0
55.0
55.0
42.5
30.0
17.5
17.5
17.5
17.5
55.0
55.0
60.0
60.0
60.0
60.0
60.0
60.0
60.0
60.0
60.0
60.0
60.0
60.0
60.0
60.0
60.0
60.0
60.0
60.0
60.0
60.0
60.0
60.0
60.0
60.0
60.0
60.0
60.0
60.0
60.0
60.0
60.0
60.0
60.0
60.0
60.0
60.0
60.0
60.0
60.0
60.0
60.0
60.0
60.0
60.0
60.0
60.0
60.0
60.0
60.0
60.0
60.0
60.0
60.0
60.0
60.0
60.0
60.0
60.0
60.0
60.0
60.0
60.0
60.0
60.0
60.0
60.0
60.0
60.0
60.0
60.0
60.0
60.0
60.0
60.0
60.0
60.0
60.0
60.0
60.0
60.0
60.0
60.0
60.0
60.0
60.0
60.0
60.0
60.0
60.0
60.0
60.0
60.0
60.0
60.0
55.0
55.0
55.0
55.0
60.0
60.0
60.0
60.0
60.0
60.0
60.0
60.0
60.0
60.0
60.0
60.0
60.0
60.0
60.0
60.0
60.0
60.0
60.0
60.0
60.0
60.0
60.0
60.0
60.0
60.0
60.0
60.0
60.0
60.0
60.0
60.0
60.0
60.0
60.0
60.0
60.0
60.0
60.0
60.0
60.0
60.0
60.0
60.0
55.0
55.0
55.0
55.0
55.0
55.0
55.0
55.0
55.0
55.0
42.5
30.0
17.5
17.5
17.5
17.5
42.5
42.5
42.5
42.5
42.5
42.5
42.5
42.5
42.5
42.5
30.

60.0
60.0
60.0
60.0
60.0
60.0
60.0
60.0
60.0
60.0
60.0
60.0
60.0
60.0
60.0
60.0
60.0
60.0
60.0
60.0
60.0
60.0
60.0
60.0
60.0
60.0
60.0
60.0
55.0
55.0
55.0
55.0
55.0
55.0
55.0
55.0
55.0
55.0
42.5
30.0
17.5
17.5
17.5
17.5
60.0
60.0
60.0
60.0
60.0
60.0
60.0
60.0
60.0
60.0
60.0
60.0
60.0
60.0
60.0
60.0
60.0
60.0
60.0
60.0
60.0
60.0
60.0
60.0
60.0
60.0
60.0
60.0
60.0
60.0
60.0
60.0
60.0
60.0
60.0
60.0
60.0
60.0
60.0
60.0
60.0
60.0
60.0
60.0
60.0
60.0
60.0
60.0
60.0
60.0
60.0
60.0
60.0
60.0
60.0
60.0
60.0
60.0
60.0
60.0
60.0
60.0
60.0
60.0
60.0
60.0
60.0
60.0
60.0
60.0
60.0
60.0
60.0
60.0
60.0
60.0
60.0
60.0
60.0
60.0
60.0
60.0
60.0
60.0
60.0
60.0
60.0
60.0
60.0
60.0
60.0
60.0
60.0
60.0
60.0
60.0
60.0
60.0
60.0
60.0
60.0
60.0
60.0
60.0
60.0
60.0
60.0
60.0
60.0
60.0
60.0
60.0
55.0
55.0
55.0
55.0
60.0
60.0
60.0
60.0
60.0
60.0
60.0
60.0
60.0
60.0
60.0
60.0
60.0
60.0
60.0
60.0
60.0
60.0
60.0
60.0
60.0
60.0
60.0
60.0
60.0
60.0
60.0
60.0
60.0
60.0
60.0
60.0
60.0
60.0
60.0
60.0
60.0
60.0
60.0
60.0


60.0
60.0
60.0
60.0
60.0
55.0
55.0
55.0
55.0
55.0
55.0
60.0
60.0
60.0
60.0
60.0
60.0
60.0
60.0
60.0
60.0
42.5
42.5
42.5
42.5
42.5
42.5
42.5
42.5
30.0
17.5
5.0
5.0
5.0
5.0
5.0
5.0
55.0
55.0
55.0
55.0
55.0
55.0
55.0
55.0
55.0
55.0
42.5
30.0
17.5
17.5
17.5
17.5
60.0
60.0
60.0
60.0
60.0
60.0
60.0
60.0
60.0
60.0
60.0
60.0
60.0
60.0
60.0
60.0
60.0
60.0
60.0
60.0
60.0
60.0
60.0
60.0
60.0
60.0
60.0
60.0
60.0
60.0
60.0
60.0
60.0
60.0
60.0
60.0
60.0
60.0
60.0
60.0
60.0
60.0
60.0
60.0
60.0
60.0
60.0
60.0
60.0
60.0
60.0
60.0
60.0
60.0
60.0
60.0
60.0
60.0
60.0
60.0
60.0
60.0
60.0
60.0
60.0
60.0
60.0
60.0
60.0
60.0
60.0
60.0
60.0
60.0
60.0
60.0
60.0
60.0
60.0
60.0
60.0
60.0
60.0
60.0
60.0
60.0
60.0
60.0
60.0
60.0
60.0
60.0
60.0
60.0
60.0
60.0
60.0
60.0
60.0
60.0
60.0
60.0
60.0
60.0
60.0
60.0
60.0
60.0
60.0
60.0
60.0
60.0
60.0
60.0
60.0
60.0
60.0
60.0
60.0
60.0
60.0
60.0
60.0
60.0
60.0
60.0
60.0
60.0
60.0
60.0
60.0
60.0
60.0
60.0
60.0
60.0
60.0
60.0
60.0
60.0
60.0
60.0
60.0
60.0
60.0
60.0
60.0
60.0
6

5.0
5.0
5.0
5.0
5.0
5.0
5.0
5.0
5.0
5.0
5.0
5.0
55.0
55.0
55.0
55.0
55.0
55.0
55.0
55.0
42.5
30.0
17.5
5.0
5.0
5.0
5.0
5.0
60.0
60.0
60.0
60.0
60.0
60.0
60.0
60.0
60.0
60.0
60.0
60.0
60.0
60.0
60.0
60.0
55.0
55.0
60.0
60.0
60.0
60.0
60.0
60.0
60.0
60.0
60.0
60.0
60.0
60.0
60.0
60.0
55.0
55.0
60.0
60.0
60.0
60.0
60.0
60.0
60.0
60.0
60.0
60.0
60.0
60.0
60.0
60.0
55.0
55.0
60.0
60.0
60.0
60.0
60.0
60.0
47.5
35.0
22.5
10.0
5.0
5.0
5.0
5.0


In [37]:
grid_cost =  battery_site1.getGridCost(output1, import_price_value, export_price_value, request_status, import_vector1,excess_vector1)

[2.781167]
[2.496648]
[1.864308]
[1.66468775]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.47859925]
[1.4441985]
[0.47859925]
[0.]
[0.]
[0.]
[0.]
[0.]
[20.16351619]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[42.51418421]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[32.30612865]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[128.1927024]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[119.7592956]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[74.73605318]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[17.96200467]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[71.34311174]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[23.2550325]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[

[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[91.46647182]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[117.76465661]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[81.4934043]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[6.40756163]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[66.90858989]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[5.32229561]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[74.28954142]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[25.9773598]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[9.78139055]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[31.30898416]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[22.85068202]
[0.]
[0.]
[0.]
[0.]
[0.]


[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[96.07084357]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[96.8245509]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[72.21226582]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[1.3478355]
[3.281229]
[1.3191145]
[0.]
[0.]
[0.]
[0.]
[0.]
[1.2783078]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.0057165]
[0.576471]
[0.0049395]
[0.]
[0.]
[0.]
[4.033302]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[35.58731208]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[8.81267382]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[69.3154161]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[51.5614932]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[75.216078]
[0.]
[0.]
[0.]
[

[0.]
[5.11998244]
[8.45450747]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[4.1847]
[3.479616]
[2.45199875]
[1.3451575]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.0930155]
[0.8503355]
[0.08155325]
[0.]
[0.]
[0.]
[0.]
[0.3385095]
[1.30875]
[2.0146735]
[2.8090935]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[7.56408608]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[1.6345095]
[0.089154]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[2.38395]
[6.20172]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.5733]
[1.872065]
[3.177152]
[4.77447]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[2.0618705]
[4.769741]
[2.09697975]
[0.]
[0.]
[0.]
[1.209295]
[0.367464]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[1.65215325]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[7.01272425]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[8.622558]
[0.]
[0.]
[0.]